In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [2]:
df = pd.read_csv(r"C:\Users\22303\Desktop\6013小组作业\dataset.csv")

In [3]:
# 分离特征（X）和标签（y）
X = df.drop('loan_status', axis=1)  # 特征列
y = df['loan_status']  # 标签列

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=114514, stratify=y)

# 转换为 LightGBM Dataset
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)  # reference 用于关联训练集（可选）

In [4]:
# 二分类核心参数（重点：objective='binary'，metric选二分类专用指标）
params = {
    'objective': 'binary',  # 任务类型：二分类
    'metric': ['auc', 'binary_logloss'],  # 评估指标：AUC（排序能力）+ 二分类对数损失
    'boosting_type': 'gbdt',  # 梯度提升决策树（默认最优）
    'learning_rate': 0.1,  # 学习率（步长）
    'num_leaves': 31,  # 每棵树最大叶子数（控制复杂度，二分类建议15~63）
    'max_depth': -1,  # 树深度（-1不限制，防止过拟合可设3~7）
    'reg_alpha': 0.1,  # L1正则（抑制过拟合）
    'reg_lambda': 0.1,  # L2正则（抑制过拟合）
    'verbose': 1,  # 日志输出级别（1=简要输出）
    'seed': 42,  # 随机种子（保证可复现）
    'is_unbalance': True  # 是否处理类别不平衡（此处数据集较平衡，设为False；不平衡时设为True）
}

# 训练模型（早停机制防止过拟合）
model = lgb.train(
    params,
    train_set=lgb_train,  # 替换为转换后的 Dataset
    num_boost_round=200,
    valid_sets=[lgb_test],  # 替换为转换后的 Dataset
    callbacks=[lgb.log_evaluation(10)]
)

[LightGBM] [Info] Number of positive: 232053, number of negative: 1570803
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043401 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 1802856, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.128714 -> initscore=-1.912376
[LightGBM] [Info] Start training from score -1.912376
[10]	valid_0's auc: 0.726729	valid_0's binary_logloss: 0.476648
[20]	valid_0's auc: 0.7296	valid_0's binary_logloss: 0.555832
[30]	valid_0's auc: 0.731772	valid_0's binary_logloss: 0.588829
[40]	valid_0's auc: 0.733333	valid_0's binary_logloss: 0.600643
[50]	valid_0's auc: 0.734569	valid_0's binary_logloss: 0.604278
[60]	valid_0's auc: 0.735323	valid_0's binary_logloss: 0.605212
[70]	valid_0's auc: 0.736362	valid_0's binary_logloss: 0.

In [5]:
# 预测：返回正类（1类）概率（二分类输出shape=(n_samples, 1)）
y_pred_prob = model.predict(X_test, num_iteration=model.best_iteration)
y_pred = (y_pred_prob > 0.5).astype(int)  # 概率阈值0.5转换为类别标签（可根据需求调整阈值）

# 核心评估指标
accuracy = accuracy_score(y_test, y_pred)  # 准确率
precision = precision_score(y_test, y_pred)  # 精确率（预测为正类的正确率）
recall = recall_score(y_test, y_pred)  # 召回率（真实正类被预测出的比例）
f1 = f1_score(y_test, y_pred)  # F1分数（精确率+召回率平衡）
auc = roc_auc_score(y_test, y_pred_prob)  # AUC值（排序能力，越接近1越好）

# 输出评估结果
print("\n===== 二分类模型评估结果 =====")
print(f"准确率（Accuracy）：{accuracy:.4f}")
print(f"精确率（Precision）：{precision:.4f}")
print(f"召回率（Recall）：{recall:.4f}")
print(f"F1分数：{f1:.4f}")
print(f"AUC值：{auc:.4f}")


===== 二分类模型评估结果 =====
准确率（Accuracy）：0.6435
精确率（Precision）：0.2241
召回率（Recall）：0.7187
F1分数：0.3417
AUC值：0.7407


In [6]:
# LightGBM模型指标与实验结果评估
# 1. 整体模型性能：AUC达到0.7407，具备中等偏上的正负样本区分能力，
#    在二分类贷款状态预测任务中表现合格。

# 2. 核心分类指标分析：
#    - 准确率（Accuracy）为0.6435，受数据不平衡影响未达到较高水平；
#    - 召回率（Recall）高达0.7187，能有效捕捉大部分实际正类样本，
#      适合需优先识别潜在风险的贷款场景；
#    - 精确率（Precision）仅0.2241，说明预测为正类的样本中实际正类占比低，
#      存在较多假阳性预测；
#    - F1分数0.3417，受低精确率拖累，反映精确率与召回率的平衡效果欠佳。

# 3. 训练过程表现：
#    - 训练200轮期间，AUC从0.7267稳步提升至0.7407，后期提升幅度逐渐收窄，
#      模型趋于收敛；
#    - 二分类对数损失（binary_logloss）在10轮时达到最低0.4766，
#      后续持续上升至0.6004，提示模型存在轻微过拟合倾向，
#      过多轮次训练导致对测试集拟合效果下降。

# 4. 数据与参数适配性：
#    - 数据存在明显不平衡（正样本占比12.87%），模型启用is_unbalance=True参数，
#      有效提升了召回率，但牺牲了部分精确率；
#    - 特征数量仅9个，模型仍能达到0.74+的AUC，说明核心特征区分度较好。

# 5. 总结与建议：
#    模型在贷款状态预测中能有效识别高风险样本，适合风险防控导向的场景；
#    需优化精确率，可通过调整阈值、增加正则化强度或特征工程进一步提升性能。